In [1]:
import os
import numpy as np
import cv2
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory paths
dataset_path = "C:/Users/eliez/Desktop/python proj/LiverScan/"
test_path = os.path.join(dataset_path, "test1")
train_path = os.path.join(dataset_path, "train1")
valid_path = os.path.join(dataset_path, "valid1/")

# Directory paths for train, test, and validation sets
test_image_path = os.path.join(test_path, "images")
test_mask_path = os.path.join(test_path, "masks")
train_image_path = os.path.join(train_path, "images")
train_mask_path = os.path.join(train_path, "masks")
valid_image_path = os.path.join(valid_path, "images")
valid_mask_path = os.path.join(valid_path, "masks")

# List of image filenames for train, test, and validation sets
test_ids = os.listdir(test_image_path)
train_ids = os.listdir(train_image_path)
valid_ids = os.listdir(valid_image_path)

image_size = 128

# Function to load and preprocess images and masks
def load_images_and_masks(image_path, mask_path, image_size):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (image_size, image_size))
    image = image / 255.0  # Normalize to [0, 1]

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (image_size, image_size))
    mask = np.expand_dims(mask, axis=-1)
    mask = mask / 255.0  # Normalize to [0, 1]

    return image, mask

# Load and preprocess the training data
X_train = []
Y_train = []

for image_id in train_ids:
    image_path = os.path.join(train_image_path, image_id)
    mask_path = os.path.join(train_mask_path, image_id)
    image, mask = load_images_and_masks(image_path, mask_path, image_size)
    X_train.append(image)
    Y_train.append(mask)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

# Define and compile the ResUNet model
def bn_act(x, act=True):
    x = keras.layers.BatchNormalization()(x)
    if act == True:
        x = keras.layers.Activation("relu")(x)
    return x

def conv_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = bn_act(x)
    conv = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
    return conv

def stem(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    conv = conv_block(conv, filters, kernel_size=kernel_size, padding=padding, strides=strides)

    shortcut = keras.layers.Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)

    output = keras.layers.Add()([conv, shortcut])
    return output

def residual_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    res = conv_block(x, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    res = conv_block(res, filters, kernel_size=kernel_size, padding=padding, strides=1)

    shortcut = keras.layers.Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)

    output = keras.layers.Add()([shortcut, res])
    return output

def upsample_concat_block(x, xskip):
    u = keras.layers.UpSampling2D((2, 2))(x)
    c = keras.layers.Concatenate()([u, xskip])
    return c

# ...

# Define and compile the ResUNet model
def ResUNet():
    f = [16, 32, 64, 128, 256]
    inputs = keras.layers.Input((image_size, image_size, 3))

    ## Encoder
    e0 = inputs
    e1 = stem(e0, f[0])
    e2 = residual_block(e1, f[1], strides=2)
    e3 = residual_block(e2, f[2], strides=2)
    e4 = residual_block(e3, f[3], strides=2)
    e5 = residual_block(e4, f[4], strides=2)

    ## Bridge
    b0 = conv_block(e5, f[4], strides=1)
    b1 = conv_block(b0, f[4], strides=1)

    ## Decoder
    u1 = upsample_concat_block(b1, e4)
    d1 = residual_block(u1, f[4])

    u2 = upsample_concat_block(d1, e3)
    d2 = residual_block(u2, f[3])

    u3 = upsample_concat_block(d2, e2)
    d3 = residual_block(u3, f[2])

    u4 = upsample_concat_block(d3, e1)
    d4 = residual_block(u4, f[1])

    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(d4)  # Change output to 1 channel
    model = keras.models.Model(inputs, outputs)
    return model



smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    return 1.0 - dice_coef(y_true, y_pred)

model = ResUNet()
adam = keras.optimizers.Adam()
model.compile(optimizer=adam, loss=dice_coef_loss, metrics=[dice_coef])

# Train the model
model.fit(X_train, Y_train, batch_size=16, epochs=20, validation_split=0.1)

# Save the trained model to an H5 file
model.save("liver_segmentation_model.h5")

Epoch 1/20
14/14 [==============================] - 48s 3s/step - loss: 0.9998 - dice_coef: 2.0453e-04 - val_loss: 1.0000 - val_dice_coef: 1.6254e-05
Epoch 2/20
 3/14 [=====>........................] - ETA: 34s - loss: 0.9935 - dice_coef: 0.0065

KeyboardInterrupt: 